In [ ]:
#https://gitlab.liris.cnrs.fr/coregraphie/netbone/-/blob/main/examples/example.ipynb
import networkx as nx
import matplotlib.pyplot as plt
import re
def parse_dot_file_manual(file_path):
    """
    Manually parse a DOT file and create a NetworkX graph.
    This method doesn't rely on external DOT parsers.
    Handles multi-line attribute definitions properly.
    
    Args:
        file_path (str): Path to the DOT file
        
    Returns:
        networkx.Graph or networkx.DiGraph: The loaded graph
    """
    try:
        with open(file_path, 'r') as f:
            content = f.read()
        
        # Remove comments
        content = re.sub(r'//.*?\n', '\n', content)
        content = re.sub(r'/\*.*?\*/', '', content, flags=re.DOTALL)
        
        # Determine if it's directed or undirected
        is_directed = 'digraph' in content.lower() or '->' in content
        
        # Create appropriate graph type
        if is_directed:
            graph = nx.DiGraph()
        else:
            graph = nx.Graph()
        
        # Tokenize the content to handle multi-line structures
        tokens = _tokenize_dot_content(content)
        
        i = 0
        while i < len(tokens):
            token = tokens[i]
            
            # Skip graph declaration and braces
            if token.lower() in ['graph', 'digraph', '{', '}'] or not token.strip():
                i += 1
                continue
            
            # Check if this is an edge (look ahead for edge operators)
            if i + 2 < len(tokens) and tokens[i + 1] in ['--', '->']:
                source = token
                edge_op = tokens[i + 1]
                target = tokens[i + 2]
                i += 3
                
                # Parse edge attributes
                edge_attrs = {}
                if i < len(tokens) and tokens[i] == '[':
                    attr_tokens, i = _parse_attributes(tokens, i)
                    edge_attrs = _parse_attribute_pairs(attr_tokens)
                
                # Convert numeric values
                for key, value in edge_attrs.items():
                    if _is_numeric(value):
                        edge_attrs[key] = float(value)
                
                graph.add_edge(source, target, **edge_attrs)
                
            # Otherwise, it's a node
            else:
                node = token
                i += 1
                
                # Parse node attributes
                node_attrs = {}
                if i < len(tokens) and tokens[i] == '[':
                    attr_tokens, i = _parse_attributes(tokens, i)
                    node_attrs = _parse_attribute_pairs(attr_tokens)
                
                # Convert numeric values for node attributes
                for key, value in node_attrs.items():
                    if _is_numeric(value):
                        node_attrs[key] = float(value)
                
                graph.add_node(node, **node_attrs)
        
        print(f"Successfully parsed DOT file: {file_path}")
        print(f"Graph type: {type(graph).__name__}")
        print(f"Number of nodes: {graph.number_of_nodes()}")
        print(f"Number of edges: {graph.number_of_edges()}")
        
        return graph
        
    except Exception as e:
        print(f"Error parsing DOT file: {e}")
        return None

def _tokenize_dot_content(content):
    """Tokenize DOT content handling quoted strings and special characters."""
    tokens = []
    i = 0
    
    while i < len(content):
        char = content[i]
        
        # Skip whitespace
        if char.isspace():
            i += 1
            continue
        
        # Handle quoted strings
        if char == '"':
            quote_end = i + 1
            while quote_end < len(content) and content[quote_end] != '"':
                if content[quote_end] == '\\':  # Handle escaped characters
                    quote_end += 1
                quote_end += 1
            if quote_end < len(content):
                tokens.append(content[i+1:quote_end])  # Remove quotes
                i = quote_end + 1
            else:
                i += 1
        
        # Handle special characters
        elif char in '[]{}=;':
            tokens.append(char)
            i += 1
        
        # Handle edge operators
        elif char == '-':
            if i + 1 < len(content) and content[i + 1] == '-':
                tokens.append('--')
                i += 2
            elif i + 1 < len(content) and content[i + 1] == '>':
                tokens.append('->')
                i += 2
            else:
                i += 1
        
        # Handle regular tokens (identifiers, numbers)
        else:
            token_start = i
            while i < len(content) and not content[i].isspace() and content[i] not in '[]{}=;-"':
                i += 1
            if i > token_start:
                tokens.append(content[token_start:i])
    
    return tokens

def _parse_attributes(tokens, start_index):
    """Parse attribute list starting from '[' and return tokens and end index."""
    if tokens[start_index] != '[':
        return [], start_index
    
    attr_tokens = []
    i = start_index + 1
    bracket_count = 1
    
    while i < len(tokens) and bracket_count > 0:
        token = tokens[i]
        if token == '[':
            bracket_count += 1
        elif token == ']':
            bracket_count -= 1
        
        if bracket_count > 0:
            attr_tokens.append(token)
        
        i += 1
    
    return attr_tokens, i

def _parse_attribute_pairs(attr_tokens):
    """Parse attribute tokens into key-value pairs."""
    attrs = {}
    i = 0
    
    while i < len(attr_tokens):
        if i + 2 < len(attr_tokens) and attr_tokens[i + 1] == '=':
            key = attr_tokens[i]
            value = attr_tokens[i + 2]
            attrs[key] = value
            i += 3
        else:
            i += 1
    
    return attrs

def _is_numeric(value):
    """Check if a string represents a numeric value."""
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def write_networkx_to_dot_manual(graph, filename):
    """
    Manual implementation to write NetworkX graph to DOT format.
    This works without requiring pygraphviz.
    
    Parameters:
    -----------
    graph : networkx.Graph
        The NetworkX graph to write
    filename : str
        Output filename
    """
    with open(filename, 'w') as f:
        # Determine if graph is directed
        if graph.is_directed():
            f.write("digraph G {\n")
            edge_sep = " -> "
        else:
            f.write("graph G {\n")
            edge_sep = " -- "
        
        # Write nodes with attributes
        for node in graph.nodes():
            attrs = graph.nodes[node]
            if attrs:
                attr_str = ", ".join([f'{k}="{v}"' for k, v in attrs.items()])
                f.write(f'  "{node}" [{attr_str}];\n')
            else:
                f.write(f'  "{node}";\n')
        
        # Write edges with attributes
        for u, v in graph.edges():
            attrs = graph.edges[u, v]
            if attrs:
                attr_str = ", ".join([f'{k}="{v}"' for k, v in attrs.items()])
                f.write(f'  "{u}"{edge_sep}"{v}" [{attr_str}];\n')
            else:
                f.write(f'  "{u}"{edge_sep}"{v}";\n')
        
        f.write("}\n")
    
    print(f"Graph successfully written to {filename}")

In [15]:
import networkx as nx
import matplotlib.pyplot as plt
import pydot

        # Read the DOT file using NetworkX
G = parse_dot_file_manual('../graph/subgraph_10.dot')

Successfully parsed DOT file: ../graph/subgraph_10.dot
Graph type: Graph
Number of nodes: 501
Number of edges: 15362


In [16]:
n = sum(nx.get_edge_attributes(G, name='weight').values())
u=0

In [17]:
import netbone as nb
from netbone.filters import boolean_filter, threshold_filter, fraction_filter
# Apply filters



In [ ]:
# Remove all edges from hss_backbone.graph where 'in_backbone' == False
hss_backbone = nb.high_salience_skeleton(G)
backbone = boolean_filter(hss_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_hss_backbone.dot')

High Salience Skeleton Filter
Graph successfully written to ../graph/subgraph_10_hss_backbone.dot


In [ ]:
ds_backbone = nb.doubly_stochastic(G)
backbone = boolean_filter(ds_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_ds_backbone.dot')

Doubly Stochastic Filter
Graph successfully written to ../graph/subgraph_10_ds_backbone.dot


In [ ]:
h_backbone = nb.h_backbone(G)
backbone = boolean_filter(h_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_h_backbone.dot')

H-Backbone Filter
Graph successfully written to ../graph/subgraph_10_h_backbone.dot


In [ ]:
mst_backbone = nb.maximum_spanning_tree(G)
backbone = boolean_filter(mst_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_mst_backbone.dot')

Maximum Spanning Tree
Graph successfully written to ../graph/subgraph_10_mst_backbone.dot


In [26]:
md_backbone = nb.metric_distance_backbone(G)
backbone = boolean_filter(md_backbone)
# Copy node labels from G to backbone
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_md_backbone.dot')

Metric Distance Filter
Graph successfully written to ../graph/subgraph_10_md_backbone.dot


In [ ]:
umd_backbone = nb.ultrametric_distance_backbone(G)
backbone = boolean_filter(umd_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_umd_backbone.dot')

Ultrametric Distance Filter
Graph successfully written to ../graph/subgraph_10_umd_backbone.dot


In [33]:
G = parse_dot_file_manual('../graph/subgraph_19.dot')
pmfg_backbone = nb.pmfg(G)
backbone = boolean_filter(pmfg_backbone)
for node in backbone.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        backbone.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(backbone, '../graph/subgraph_19_pmfg_backbone.dot')

Successfully parsed DOT file: ../graph/subgraph_19.dot
Graph type: Graph
Number of nodes: 480
Number of edges: 13640
Citation here
Graph successfully written to ../graph/subgraph_19_pmfg_backbone.dot


In [ ]:
# plam_backbone = nb.plam(G)
# backbone = boolean_filter(plam_backbone)
# write_networkx_to_dot_manual(backbone, '../graph/subgraph_10_pla_backbone.dot')

In [ ]:

nc_backbone = nb.noise_corrected(G,False)
edges_to_remove = [(u, v) for u, v, d in nc_backbone.graph.edges(data=True) if float(d.get('score')) <0.1]
nc_backbone.graph.remove_edges_from(edges_to_remove)
for node in nc_backbone.graph.nodes():
    if node in G.nodes and 'label' in G.nodes[node]:
        nc_backbone.graph.nodes[node]['label'] = G.nodes[node]['label']
write_networkx_to_dot_manual(nc_backbone.graph, '../graph/subgraph_10_nc_backbone.dot')

Graph successfully written to ../graph/subgraph_7_nc_backbone.dot
